# Hover-Net Example

## Imports

In [ ]:
# System
import sys
import os

# Misc
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# ML
import torch
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

# Augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Custom
sys.path.append(os.path.abspath(".."))
from dataloader.train_loader import PumaDataset

In [ ]:
IMAGE_DIR = '../data/01_training_dataset_tif_ROIs'
GEOJSON_DIR = '../data/01_training_dataset_geojson_nuclei'

transform = A.Compose([
    A.RandomCrop(width=512, height=512),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    ToTensorV2()
])

dataset = PumaDataset(image_dir=IMAGE_DIR, geojson_dir=GEOJSON_DIR, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

for images, masks in dataloader:
    print(images.shape, masks.shape)
    break

plot = True
if plot:
    fig, axs = plt.subplots(3, 2, figsize=(10, 15))
    idx = 0
    for images, masks in dataloader:
        axs[idx, 0].imshow(images[0].permute(1, 2, 0))
        axs[idx, 1].imshow(masks[0])
        idx += 1
        if idx == 3:
            break